In [ ]:
%pip install -U starpoint openai sentence-transformers tokenizers

In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
STARPOINT_API_KEY = os.environ["STARPOINT_API_KEY"]

In [ ]:
def embed(texts):
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(texts)

def prompt_openai(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )["choices"][0]["message"]["content"]


In [ ]:
import json
import requests
from starpoint.embedding import EmbeddingClient, EmbeddingModel
from tokenizers import Tokenizer

embedding_client = EmbeddingClient(api_key=STARPOINT_API_KEY)

EMBEDDING_URL = "https://scholar.starpoint.ai/api/v1/embed"

tokenizer = Tokenizer.from_pretrained("bert-base-cased")

with open('./monster_text.json') as f:
    monster_texts = json.load(f)

documents_to_upload = []
embeddings = embed([monster["text"] for monster in monster_texts])

for index, monster in enumerate(monster_texts):
    documents_to_upload.append({
        "embedding": embeddings[index].tolist(),
        "metadata": monster
    })
print(documents_to_upload[:10])

In [ ]:
import copy
from starpoint.db import Client

COLLECTION_NAME = "dnd"

client = Client(api_key=STARPOINT_API_KEY)
client.insert(documents=documents_to_upload, collection_name=COLLECTION_NAME)

In [ ]:
import requests

EMBEDDING_URL = "https://scholar.starpoint.ai/api/v1/embed"
headers = {"x-starpoint-key": STARPOINT_API_KEY}

question = "how big is an aerosaur?"
embed_resp = embed([question])

# query_embedding = embed_resp.json()["results"][0]["embedding"]
query_embedding = embed_resp[0]



In [ ]:
from sentence_transformers import SentenceTransformer
# from llama_cpp import Llama
from starpoint.db import Client

COLLECTION_ID = '21643fcc-8620-49db-abb2-8b18116567f4'
starpoint_client = Client(api_key=STARPOINT_API_KEY)
GGML_MODEL_PATH = "./model/model.bin"

# llm = Llama(model_path=GGML_MODEL_PATH)

def query_starpoint(text):
    query_embedding = model.encode(text).tolist()
    relevant_monsters = starpoint_client.query(
        collection_id=COLLECTION_ID,
        query_embedding=query_embedding,
        sql="SELECT * FROM collection LIMIT 10"
    )
    return relevant_monsters


In [ ]:
resp = query_starpoint("where are Balors found?")
print(resp)